In [7]:
import os
import cv2
import numpy as np
import random
import tensorflow as tf
from collections import deque

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [8]:
dataset_path = "C:/Users/batuh/Downloads/Compressed/NTU_Dataset_2"
actions = os.listdir("C:/Users/batuh/Downloads/Compressed/NTU_Dataset_2")

In [9]:
frame_count = 20

frame_width = 64
frame_height = 64

seed_constant = 86
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [10]:
def take_video_names(dataset_path):

  actions = os.listdir(dataset_path)
  video_paths = []
  video_labels = []

  for single_action in actions:

    videos = os.listdir(dataset_path + "/" + single_action)

    for single_video in videos:

      single_video_path = dataset_path + "/" + single_action + "/" + single_video

      video_paths.append(single_video_path)
      video_labels.append(single_action)

  print(f"There is {len(actions)} action in the dataset.")
  print(f"There is {len(video_paths)} video in the dataset\n")

  return np.array(video_paths), np.array(video_labels)

In [11]:
def video_preprocess(frame):

  resized_frame = cv2.resize(frame, (frame_height, frame_width))

  normalized_frame = resized_frame / 255

  return normalized_frame

In [12]:
def load_video (video_path):

  frames_list = []

  cap = cv2.VideoCapture(video_path)

  ok, frame = cap.read()

  video_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

  skip_ratio = max(int(video_frame_count/frame_count), 1)

  for i in range(frame_count):

    cap.set(cv2.CAP_PROP_POS_FRAMES, i * skip_ratio)

    oke, frame = cap.read()

    if not oke:
      break

    frame = video_preprocess(frame)

    frames_list.append(frame)

  if len(frames_list) != frame_count:
    print(f"There is not enough number of frames in this video {video_path}")

  cap.release()

  return frames_list

In [13]:
feature_s = []

count = 0

videos, labels = take_video_names(dataset_path)

for one_video in videos:

  count = count + 1

  feature_s.append(load_video(one_video))

  print (f"{count/len(videos)*100} percent complete ")

feature_s = np.array(feature_s)

There is 4 action in the dataset.
There is 1280 video in the dataset

0.078125 percent complete 
0.15625 percent complete 
0.234375 percent complete 
0.3125 percent complete 
0.390625 percent complete 
0.46875 percent complete 
0.546875 percent complete 
0.625 percent complete 
0.703125 percent complete 
0.78125 percent complete 
0.8593750000000001 percent complete 
0.9375 percent complete 
1.015625 percent complete 
1.09375 percent complete 
1.171875 percent complete 
1.25 percent complete 
1.328125 percent complete 
1.40625 percent complete 
1.484375 percent complete 
1.5625 percent complete 
1.640625 percent complete 
1.7187500000000002 percent complete 
1.7968749999999998 percent complete 
1.875 percent complete 
1.953125 percent complete 
2.03125 percent complete 
2.109375 percent complete 
2.1875 percent complete 
2.265625 percent complete 
2.34375 percent complete 
2.421875 percent complete 
2.5 percent complete 
2.578125 percent complete 
2.65625 percent complete 
2.734375 perc

In [14]:
print(len(feature_s))
print(type(feature_s))

1280
<class 'numpy.ndarray'>


In [15]:
le = LabelEncoder()
labels = le.fit_transform(labels)

one_hot_encoded_labels = to_categorical(labels)

In [16]:
features_train, features_test, labels_train, labels_test = train_test_split(feature_s, 
                                                                            one_hot_encoded_labels, 
                                                                            test_size = 0.25, 
                                                                            shuffle = True, 
                                                                            random_state = seed_constant)

In [17]:
del feature_s
del labels
del count

In [18]:
def create_LRCN_model():

    model = Sequential()
    
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (frame_count, frame_height, frame_width, 3)))
    
    model.add(TimeDistributed(MaxPooling2D((4, 4)))) 
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
                                      
    model.add(TimeDistributed(Flatten()))
                                      
    model.add(LSTM(32))
                                      
    model.add(Dense(len(actions), activation = 'softmax'))


    model.summary()
    
    return model

In [19]:
LRCN_model = create_LRCN_model()

print("Model Created Successfully!")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 20, 64, 64, 16)    448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 16, 16, 16)    0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 16, 16, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 16, 16, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 4, 4, 32)      0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 4, 4, 32)      0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 4, 4, 64)      1

In [20]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 60, mode = 'min', restore_best_weights = True)

LRCN_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

LRCN_model_training_history = LRCN_model.fit(x = features_train, y = labels_train, epochs = 70, batch_size = 4 ,
                                             shuffle = True, validation_split = 0.2, 
                                             callbacks = [early_stopping_callback])

Epoch 1/70
192/192 [==============================] - 7s 13ms/step - loss: 1.4061 - accuracy: 0.2474 - val_loss: 1.3891 - val_accuracy: 0.2604
Epoch 2/70
192/192 [==============================] - 2s 9ms/step - loss: 1.3881 - accuracy: 0.2591 - val_loss: 1.3848 - val_accuracy: 0.2969
Epoch 3/70
192/192 [==============================] - 2s 8ms/step - loss: 1.3879 - accuracy: 0.2487 - val_loss: 1.3872 - val_accuracy: 0.2396
Epoch 4/70
192/192 [==============================] - 2s 8ms/step - loss: 1.3879 - accuracy: 0.2474 - val_loss: 1.3864 - val_accuracy: 0.2396
Epoch 5/70
192/192 [==============================] - 2s 8ms/step - loss: 1.3880 - accuracy: 0.2539 - val_loss: 1.3856 - val_accuracy: 0.2396
Epoch 6/70
192/192 [==============================] - 2s 8ms/step - loss: 1.3836 - accuracy: 0.2812 - val_loss: 1.3623 - val_accuracy: 0.3594
Epoch 7/70
192/192 [==============================] - 2s 8ms/step - loss: 1.1962 - accuracy: 0.4349 - val_loss: 1.1519 - val_accuracy: 0.4323
Epoch

In [21]:
model_evaluation_history = LRCN_model.evaluate(features_test, labels_test)

10/10 [==============================] - 1s 14ms/step - loss: 0.5492 - accuracy: 0.7844


In [2]:
def predict_on_camera():

    cam = cv2.VideoCapture(0)
    
    cam_frame_list = deque(maxlen = frame_count)

    while True:

        ok, cam_frame = cam.read()
    
        if ok == True:
        
            normalized_cam_frame = video_preprocess(cam_frame)

            cam_frame_list.append(normalized_cam_frame)

        if len(cam_frame_list) == 20:

            predicted_labels_probabilities = LRCN_model.predict(np.expand_dims(cam_frame_list, axis = 0))[0]
            
            for i in range(len(actions)):
                
                if predicted_labels_probabilities[i] > 0.95 and predicted_labels_probabilities[i] < 1.0:
                    
                    predicted_label = np.argmax(predicted_labels_probabilities)

                    predicted_class_name = actions[predicted_label]

                    cv2.putText(cam_frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
        cv2.imshow("Frame", cam_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cam.release()

In [33]:
while True:
    
    predict_on_camera()

: 

: 